In [8]:
import pandas as pd

# First load & Inspect
raw = pd.read_csv('../data/timesheet.csv', parse_dates=['Date'])
raw.info()
raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          90 non-null     datetime64[ns]
 1   Name          90 non-null     object        
 2   Location      90 non-null     object        
 3   Department    49 non-null     object        
 4   Clock in      90 non-null     object        
 5   Clock out     90 non-null     object        
 6   Break         86 non-null     object        
 7   Total hrs     86 non-null     object        
 8   Schedule hrs  33 non-null     object        
 9   Status        90 non-null     object        
dtypes: datetime64[ns](1), object(9)
memory usage: 7.2+ KB


,Date,Name,Location,Department,Clock in,Clock out,Break,Total hrs,Schedule hrs,Status
0,2018-05-01,Shiva Manhar,Raipur HQ,Product,10:05,18:05,0 h 31 m,8 h 44 m,9 h 0 m,Approved
1,2018-05-01,Akash Sharma,Raipur HQ,Product,10:05,19:05,0 h 20 m,8 h 54 m,9 h 0 m,Approved
2,2018-05-01,Vipplove Vishwakarma,Raipur HQ,Product,10:05,19:05,0 h 19 m,8 h 55 m,9 h 0 m,Approved
3,2018-05-01,Akash Chandrakar,Raipur HQ,Product,10:05,21:05,0 h 54 m,11 h 21 m,12 h 0 m,Unapproved
4,2018-05-02,Shiva Manhar,Raipur HQ,Product,10:05,19:05,0 h 20 m,9 h 2 m,9 h 0 m,Approved


In [10]:
def parse_hours(hstr):
    """Convert strings from '8 h 44 m' -> 8.73333 hours (8 + 44/60).
    Handles '11 h 0 m' etc.
    """
    if pd.isna(hstr):
        return pd.NA
    parts = hstr.strip().split()
    hours = int(parts[0])
    minutes = int(parts[2])
    return hours + minutes / 60

raw["hours_worked"] = raw["Total hrs"].apply(parse_hours)

In [11]:
# Rename & filter
df = (
    raw
    .rename(columns={
        "Name": "employee_name",
    })
    .loc[lambda d: d["hours_worked"] > 0] # drop <= 0
    .dropna(subset=["hours_worked"]) # drop NA hours
)

In [13]:
# Checking my work
print(df["hours_worked"].describe())
print(df.isna().sum().sort_values(ascending=False).head(10))

count     86.00
unique    62.00
top        8.75
freq       6.00
Name: hours_worked, dtype: float64
Schedule hrs     57
Department       41
Date              0
Location          0
employee_name     0
Clock out         0
Clock in          0
Break             0
Total hrs         0
Status            0
dtype: int64


In [16]:
# Save to Parquet
df.to_parquet("../data/timesheet_clean.parquet", index=False)